# Build a reduced ETC model from RECON3D

In [3]:
from pytfa.redgem.lumpgem import LumpGEM
from pytfa.io.json import load_json_model
from cobra import Reaction
from cobra.util import Zero

recon3d = load_json_model('./../data/GEM_Recon3_thermocurated_redHUMAN.json')

2024-04-10 08:57:24,346 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [4]:
# Define the core model as all reactions that connect the ETC intermediates (RedGEM, paper = D0 ) 
# Quinones, Cytochromes, and ATP synthase

ubiquinone_reqations = [r.id for r in recon3d.metabolites.q10_m.reactions if recon3d.metabolites.q10h2_m in r.metabolites]
ferrocytochrome_reactions = [r.id for r in recon3d.metabolites.focytC_m.reactions if recon3d.metabolites.ficytC_m in r.metabolites]

core_reaction_ids = set(ubiquinone_reqations + ferrocytochrome_reactions + ['ATPS4mi'])
core_reactions = [r for r in recon3d.reactions if r.id in core_reaction_ids]

# Print a table with the core reactions
print('Core reactions')
print('--------------')
for r in core_reactions:
    print(r.id, r.reaction)



Core reactions
--------------
DHORD9 dhor_S_c + q10_m --> orot_c + q10h2_m
ETFQO etfrd_m + q10_m --> etfox_m + q10h2_m
L_LACDcm 2.0 ficytC_m + lac_L_c --> 2.0 focytC_m + 2.0 h_c + pyr_c
SULFOX 2.0 ficytC_m + h2o_c + so3_c --> 2.0 focytC_m + 2.0 h_c + so4_c
r0205 glyc3p_c + q10_m --> dhap_c + q10h2_m
r0310 pmtcoa_m + q10_m --> hdd2coa_m + q10h2_m
r0509 q10_m + succ_m --> fum_m + q10h2_m
r0541 glutcoa_m + h_m + q10_m --> b2coa_m + co2_m + q10h2_m
r0560 ibcoa_m + q10_m --> 2mp2coa_m + q10h2_m
r0603 2mbcoa_m + q10_m --> 2mb2coa_m + q10h2_m
r0655 ivcoa_m + q10_m --> 3mb2coa_m + q10h2_m
r1446 btcoa_m + q10_m --> b2coa_m + q10h2_m
r1447 ddcacoa_m + q10_m --> dd2coa_m + q10h2_m
r1448 occoa_m + q10_m --> HC01415_m + q10h2_m
r1449 q10_m + tdcoa_m --> HC01412_m + q10h2_m
r1450 hxcoa_m + q10_m --> hx2coa_m + q10h2_m
r1451 dcacoa_m + q10_m --> dc2coa_m + q10h2_m
FADH2ETC fadh2_m + q10_m --> fad_m + h_m + q10h2_m
HMR_3859 2.0 ficytC_m + lac_D_c --> 2.0 focytC_m + 2.0 h_c + pyr_c
HMR_6500 CE5021_c + 

In [5]:
###########################################
## Prepare the model lumpGEM
###########################################

# Add dummy reactions for ATP, nadh and nadph
cyt_atp2adp = Reaction('cyt_atp2adp')
cyt_nadh2nad = Reaction('cyt_nadh2nad')
recon3d.add_reactions([ cyt_atp2adp, cyt_nadh2nad])

cyt_atp2adp.reaction = 'atp_c + h2o_c --> adp_c + pi_c + h_c'
cyt_nadh2nad.reaction = 'nadh_c + h_c --> nad_c'

# This converts cobra reactions to pytfa (adds binary variables/constrains)
bound = 100
recon3d.objective = Zero # Symbolic zero objective

recon3d.prepare()
recon3d.convert()



2024-04-10 08:58:23,002 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


2024-04-10 08:58:39,261 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-04-10 08:58:39,263 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-04-10 08:58:39,264 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-04-10 08:58:39,266 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-04-10 08:58:39,269 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-04-10 08:58:39,270 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-04-10 08:58:39,272 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-04-10 08:58:39,274 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-04-10 08:58:39,313 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-04-10 08:58:39,314 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

In [6]:
recon3d.reactions.get_by_id('cyt_atp2adp').bounds = (-bound, bound)
recon3d.reactions.get_by_id('cyt_nadh2nad').bounds = (-bound, bound)



In [18]:
# By default only allow secretions for lumping + defined medium
for rxn in recon3d.boundary:
    rxn.bounds = (0, 100)

# Allow for net water and proton exchange with the environment
recon3d.reactions.get_by_id('EX_h_e').bounds = (-100.0, 100.0)
recon3d.reactions.get_by_id('EX_h2o_e').bounds = (-100.0, 100.0)
recon3d.reactions.get_by_id('EX_o2_e').bounds = (-100.0, 100.0)
recon3d.reactions.get_by_id('EX_pi_e').bounds = (-100.0, 100.0)

# Test the model 
recon3d.reactions.get_by_id('EX_lac_L_e').bounds = (-1, 100)
recon3d.reactions.get_by_id('EX_glc_D_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_bhb_e').bounds = (0, 100)
recon3d.reactions.get_by_id('EX_hdca_e').bounds = (0, 100)

recon3d.objective = recon3d.reactions.get_by_id('EX_co2_e')
recon3d.optimize()


,fluxes,reduced_costs
10FTHF5GLUtl,0.0,None
10FTHF5GLUtm,0.0,None
10FTHF6GLUtl,0.0,None
10FTHF6GLUtm,0.0,None
10FTHF7GLUtl,0.0,None
...,...,...
CYOOm2i,0.0,None
ARTPLM1,0.0,None
ARTPLM2,0.0,None
cyt_atp2adp,-100.0,None


In [20]:
# By default only allow secretions for lumping + defined medium
for rxn in recon3d.boundary:
    rxn.bounds = (0, 100)

# Allow for net water and proton exchange with the environment
recon3d.reactions.get_by_id('EX_h_e').bounds = (-100.0, 100.0)
recon3d.reactions.get_by_id('EX_h2o_e').bounds = (-100.0, 100.0)
recon3d.reactions.get_by_id('EX_o2_e').bounds = (-100.0, 100.0)
recon3d.reactions.get_by_id('EX_pi_e').bounds = (-100.0, 100.0)



In [21]:
# Generate a lump for each carbon source 
EPSILON = 1e-6

# LumpGEM parameters
params = {
    'core_subsystems': [],
    'extracellular_system': [],
    'biomass_rxns': ['EX_co2_e'],  # These are reactions that we optimize for = final product
    'timeout': 500,  # max time in s
    "constraint_method": 'both',
    # Stuff we dont need for this purpose
    "small_metabolites": [],
    "cofactor_pairs": [],
    "inorganics": [],
    "max_lumps_per_BBB": 10  # Maximal number of alternatives
}

In [22]:
# Carbon sources = lactate, glucose, 3hb, palmitate
carbon_sources = [('EX_lac_L_e',3), ('EX_glc_D_e',6), ('EX_bhb_e',4), ('EX_hdca_e',16)]


In [23]:
def find_respiration_lumps(carbon_source, n_carbon, model, params=params, 
                           core_reactions=[], method='min+1'):  
        # Lump the reactions 
        
        params['growth_rate'] = n_carbon-EPSILON

        with model as resp_model:
    
            resp_model.reactions.get_by_id(carbon_source).bounds = (-1-EPSILON, -1+EPSILON)
            resp_model.objective = resp_model.reactions.get_by_id('EX_co2_e')
            resp_model.objective_direction = 'max'
            sol = resp_model.optimize()
            print("Max CO2 production: ", sol.objective_value)
            model4lump = resp_model.copy()

            subnetwork_extraction = LumpGEM(model4lump, core_reactions, params)

            lumps = subnetwork_extraction.compute_lumps(force_solve=False, method=method)

        return lumps

In [24]:
# Find the lumps for each carbon source
lumps = {}

for carbon_source, n_carbon in carbon_sources:
    lumps[carbon_source] = find_respiration_lumps(carbon_source, n_carbon, recon3d, core_reactions=core_reactions)



Max CO2 production:  3.0


2024-04-10 09:12:08,559 - thermomodel_Recon3thermoCurated - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


Timeout limit is 500s


2024-04-10 09:13:14,772 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


...

2024-04-10 09:44:15,463 - thermomodel_Recon3thermoCurated - INFO - # Model preparation starting...


Preparing sinks...


2024-04-10 09:44:32,822 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/F
2024-04-10 09:44:32,823 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/U
2024-04-10 09:44:32,825 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-04-10 09:44:32,826 - thermomodel_Recon3thermoCurated - WARNING - Warning : C58H102N3O31FULLRCO/L
2024-04-10 09:44:32,829 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/F
2024-04-10 09:44:32,830 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/U
2024-04-10 09:44:32,832 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-04-10 09:44:32,833 - thermomodel_Recon3thermoCurated - WARNING - Warning : C92H158N5O55FULLRCO/L
2024-04-10 09:44:32,858 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52H92N3O27FULLRCO/F
2024-04-10 09:44:32,859 - thermomodel_Recon3thermoCurated - WARNING - Warning : C52

Lumping method detected: min+1


met=co2_e:   0%|          | 0/1 [00:00<?, ?it/s]

In [1]:
# Save the lumps in a json file Lumps are NamedTuple objects
import json

lumps_json = {k: [l._asdict() for l in v] for k,v in lumps.items()}
with open('lumps.json', 'w') as f:
    json.dump(lumps_json, f, indent=4)


NameError: name 'lumps' is not defined

In [ ]:

# Make a new model with the lumps only containing the core reactions and the lumps
reduced_model = recon3d.copy()

 
lump_subnet_reactions = list({r for l in lumps.values() for r in l.subnetwork})
transports = [r.id for r in recon3d.reactions if r.thermo.is_transport]

reduced_reactions = core_reactions + lump_subnet_reactions + transports

reactions_to_remove = [r for r in reduced_model.reactions if r.id not in reduced_reactions]
reduced_model.remove_reactions(reactions_to_remove)

# Make flux variability and remove reactions that are not active
reduced_model.optimize()



In [ ]:
# Generate lumps based on the overlap of the lumps

# Subent 1 {A: 1 , B: 1, C: 1}
# Subnet 2 {A: 1, G: 1, D: 1} 
# Sum the two subnets {A: 2, B: 1, C: 1, G: 1, D: 1}

def sum_subnets(subnet1, subnet2):
    sum_subnet = {}
    for k,v in subnet1.items():
        sum_subnet[k] = sum_subnet.get(k, 0) + v
    for k,v in subnet2.items():
        sum_subnet[k] = sum_subnet.get(k, 0) + v
    return sum_subnet


In [ ]:
# Make summed subnet for each carbon source
sum_subnetworks = {}

for carbon_source, n_carbon in carbon_sources:
    sum_subnetworks[carbon_source] = lumps[carbon_source][0]
    for i in range(1, len(lumps[carbon_source])):
        sum_subnetworks[carbon_source] = sum_subnets(sum_subnetworks[carbon_source], lumps[carbon_source][i])

# Save the summed subnetworks in a json file
with open('sum_subnetworks.json', 'w') as f:
    json.dump(sum_subnetworks, f, indent=4)

In [ ]:
# For each reaction in the summed subnetworks, add a list of the carbon sources that contain the reaction
reaction_sources = {}

for carbon_source, n_carbon in carbon_sources:
    for reaction, n_reaction in sum_subnetworks[carbon_source].items():
        reaction_sources[reaction] = reaction_sources.get(reaction, []) + [carbon_source]

# Make each list unique
for reaction, sources in reaction_sources.items():
    reaction_sources[reaction] = list(set(sources))

# Print the reaction sources for each reaction
print('Reaction sources')
print('----------------')
for reaction, sources in reaction_sources.items():
    # Stoichiometry 
    
    print(reaction, recon3d.reactions.get_by_id(reaction).reaction,[sum_subnetworks[source][reaction] for source in sources],  sources)


In [ ]:
# Partition the lumps based on the reaction sources and connectivity 
